In [1]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import asyncio
import logging
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import mplfinance as mpf
from matplotlib.widgets import Slider
import time
from functools import lru_cache
from sklearn.cluster import KMeans
import plotly.graph_objects as go


# Load local CSV file and parse into dataframe object
btc = pd.read_csv('BTC-USD.06282017-06282022.csv')

In [ ]:
# Convert adjusted closing price to numpy array
btc_prices = np.array(btc["Adj Close"])
print("BTC Prices:\n", btc_prices)

# Perform cluster analysis
K = 6
kmeans = KMeans(n_clusters=6).fit(btc_prices.reshape(-1, 1))

# predict which cluster each price is in
clusters = kmeans.predict(btc_prices.reshape(-1, 1))
print("Clusters:\n", clusters)

BTC Prices:
 [ 2506.469971  2518.439941  1929.819946  2730.399902  2757.179932
  3213.939941  4073.26001   4087.659912  4382.879883  4582.959961
  4122.939941  3582.879883  3682.840088  4403.740234  4610.47998
  5678.189941  6008.419922  6153.850098  7407.410156  5950.069824
  8036.490234  9330.549805 11323.200195 15455.400391 19140.800781
 13925.799805 14156.400391 16477.599609 13772.       11600.099609
 11786.299805  8277.009766  8129.970215 10551.799805  9664.730469
 11512.599609  9578.629883  8223.679688  8495.780273  6844.22998
  7023.52002   8329.110352  8802.459961  9419.080078  9654.799805
  8723.94043   8513.25      7368.220215  7720.25      6786.02002
  6499.27002   6173.22998   6385.819824  6773.879883  6359.640137
  7418.490234  8218.459961  7068.47998   6322.689941  6506.069824
  6707.259766  7272.720215  6300.859863  6517.180176  6710.629883
  6625.560059  6602.950195  6290.930176  6482.350098  6486.390137
  6376.129883  6411.27002   5623.540039  4009.969971  4139.87793
 

In [4]:

# Assigns plotly as visualization engine
pd.options.plotting.backend = 'plotly'

# Arbitrarily 6 colors for our 6 clusters
colors = ['red', 'orange', 'yellow', 'green', 'blue', 'indigo']

# Create Scatter plot, assigning each point a color based
# on it's grouping where group_number == index of color.
fig = btc.plot.scatter(
    x=btc.index,
    y="Adj Close",
    color=[colors[i] for i in clusters],
)

# Configure some styles
layout = go.Layout(
    plot_bgcolor='#efefef',
    showlegend=False,
    # Font Families
    font_family='Monospace',
    font_color='#000000',
    font_size=20,
    xaxis=dict(
        rangeslider=dict(
            visible=False
        ))
)
fig.update_layout(layout)

# Display plot in local browser window
fig.show()


In [39]:
# Create list to hold values, initialized with infinite values
cluster_min_max_price = []

# init for each cluster group
for i in range(6):

    # Add values for which no price could be greater or less
    cluster_min_max_price.append([np.inf, -np.inf])

# Print initial values
print(cluster_min_max_price)


for i in range(len(btc_prices)):
    # Get the cluster number for the current price
    cluster_number = clusters[i]
    
    if btc_prices[i] < cluster_min_max_price[cluster_number][0]:
        cluster_min_max_price[cluster_number][0] = btc_prices[i]

    if btc_prices[i] > cluster_min_max_price[cluster_number][1]:
        cluster_min_max_price[cluster_number][1] = btc_prices[i]


print(cluster_min_max_price)

[[inf, -inf], [inf, -inf], [inf, -inf], [inf, -inf], [inf, -inf], [inf, -inf]]
[[1929.819946, 7688.077148], [54771.578125, 65466.839844], [29445.957031, 41247.824219], [15455.400391, 26762.648438], [41911.601563, 51753.410156], [7720.25, 14156.400391]]


In [ ]:
import plotly.graph_objects as go

# Again, assign an arbitrary color to each of the 6 clusters
colors = ['red', 'orange', 'yellow', 'green', 'blue', 'indigo']

# Create Scatter plot, assigning each point a color where
# point group = color index.
fig = btc.plot.scatter(
    x=btc.index,
    y="Adj Close",
    color=[colors[i] for i in clusters],
)

# Add horizontal lines
for cluster_min, cluster_max in cluster_min_max_price:
    fig.add_hline(y=cluster_min, line_width=1, line_color="blue")
    fig.add_hline(y=cluster_max, line_width=1, line_color="blue")

# Add a trace of the price for better clarity
fig.add_trace(go.Trace(
    x=btc.index,
    y=btc['Adj Close'],
    line_color="black",
    line_width=1
))

# Make it pretty
layout = go.Layout(
    plot_bgcolor='#efefef',
    showlegend=False,
    # Font Families
    font_family='Monospace',
    font_color='#000000',
    font_size=20,
    xaxis=dict(
        rangeslider=dict(
            visible=False
        ))
)
fig.update_layout(layout)
fig.show()

c:\Users\ASUS\.pyenv\pyenv-win\versions\3.10.0\lib\site-packages\plotly\graph_objs\_deprecations.py:687: DeprecationWarning:

plotly.graph_objs.Trace is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.




In [156]:
intercepts = []
data_size_list = []
data_list = []
upper_lower_capped = []
for i in range(K):
    data = []
    for j in range(len(btc_prices)):
        if clusters[j] == i:
            data.append(btc_prices[j])

    data_list.append(data)
    
    upper_capped = np.percentile(data_list[i], 75)
    lower_capped = np.percentile(data_list[i], 25)

    upper_lower_capped.append({'upper':upper_capped, 'lower':lower_capped})
    

In [185]:
data_capped = [{} for i in range(K)]
for i in range(K):
    upper = data_list[i] >= upper_lower_capped[i]['upper']
    lower = data_list[i] <= upper_lower_capped[i]['lower']
    
    upper = [data_list[i][j] for j in range(len(data_list[i])) if upper[j]]
    lower = [data_list[i][j] for j in range(len(data_list[i])) if lower[j]]

    data_capped[i]['upper'] = upper
    data_capped[i]['lower'] = lower


In [186]:
data_capped

[{'upper': [7407.410156,
   6844.22998,
   7023.52002,
   7368.220215,
   6786.02002,
   6773.879883,
   7418.490234,
   7068.47998,
   7272.720215,
   6972.371582,
   7688.077148,
   7047.916992,
   7424.29248,
   7564.345215,
   7152.301758,
   7511.588867,
   7422.652832,
   7411.317383,
   6791.129395,
   6971.091797,
   7189.424805,
   7679.867188],
  'lower': [2506.469971,
   2518.439941,
   1929.819946,
   2730.399902,
   2757.179932,
   3213.939941,
   3582.879883,
   3682.840088,
   4009.969971,
   3614.234375,
   3252.839111,
   3998.980225,
   3865.952637,
   3552.953125,
   3601.013672,
   3583.96582,
   3464.013428,
   3690.188232,
   3673.836182,
   3810.42749,
   3847.175781,
   3951.599854]},
 {'upper': [61553.617188,
   60930.835938,
   61318.957031,
   63326.988281,
   65466.839844],
  'lower': [55950.746094,
   56216.183594,
   56631.078125,
   54771.578125,
   57248.457031]},
 {'upper': [38903.441406,
   39097.859375,
   39974.894531,
   38431.378906,
   41247.82421

In [188]:
for i in range(K):
    upper = data_capped[i]['upper']
    lower = data_capped[i]['lower']
    
    coeff_upper = np.polyfit(np.arange(len(upper)), upper, 1)
    coeff_lower = np.polyfit(np.arange(len(lower)), lower, 1)

    intercepts.append({'upper':coeff_upper[1], 'lower':coeff_lower[1]})


In [189]:
intercepts

[{'upper': 7089.655786660078, 'lower': 2718.485487083005},
 {'upper': 60474.92812540001, 'lower': 55933.44531280004},
 {'upper': 39378.03580708333, 'lower': 33170.84700541666},
 {'upper': 25413.830273599993, 'lower': 15766.659277400007},
 {'upper': 50111.40471517856, 'lower': 43367.02441457142},
 {'upper': 12186.234116418973, 'lower': 8216.603069873514}]

In [205]:

def loss_derivative(y, y_pred):
    err = y_pred - y
    return np.sum(err) / len(err)

def optimize_intercept(x, y, intercept):
    y_pred = 0 * x + intercept
    temp = 0
    MAX_ITERATION = 1000
    i = 1
    while i <= MAX_ITERATION:
        dj_dw = loss_derivative(y, y_pred)
        if abs(dj_dw - temp) <= 1e-5:
            break
        intercept = intercept - 0.01 * dj_dw
        y_pred = 0 * x + intercept
        temp = dj_dw
        i += 1
    return intercept

In [209]:
level = []
for i in range(K):
    btc_pred_upper = 0*np.arange(len(data_capped[i]['upper'])) + intercepts[i]['upper']
    btc_pred_lower = 0*np.arange(len(data_capped[i]['lower'])) + intercepts[i]['lower']
    
    
    support = optimize_intercept(np.arange(len(data_capped[i]['lower'])), data_capped[i]['lower'], intercepts[i]['lower'])
    resistance = optimize_intercept(np.arange(len(data_capped[i]['upper'])), data_capped[i]['upper'], intercepts[i]['upper'])
    
    level.append([support, resistance])
    
    


In [210]:
level

[[3401.748660812081, 7217.692115227887],
 [56163.59865736406, 62519.35939194146],
 [31370.581140953444, 39413.840739314655],
 [15842.035564271182, 24384.92772192432],
 [42692.85165099286, 50223.23903333595],
 [8314.467898507231, 11921.199741358563]]

In [211]:
import plotly.graph_objects as go

# Again, assign an arbitrary color to each of the 6 clusters
colors = ['red', 'orange', 'yellow', 'green', 'blue', 'indigo']

# Create Scatter plot, assigning each point a color where
# point group = color index.
fig = btc.plot.scatter(
    x=btc.index,
    y="Adj Close",
    color=[colors[i] for i in clusters],
)

fig.add_hline(y=level[0][0], line_width=1, line_color="green")
fig.add_hline(y=level[0][1], line_width=1, line_color="red")
fig.add_hline(y=level[1][0], line_width=1, line_color="green")
fig.add_hline(y=level[1][1], line_width=1, line_color="red")
fig.add_hline(y=level[2][0], line_width=1, line_color="green")
fig.add_hline(y=level[2][1], line_width=1, line_color="red")
fig.add_hline(y=level[3][0], line_width=1, line_color="green")
fig.add_hline(y=level[3][1], line_width=1, line_color="red")
fig.add_hline(y=level[4][0], line_width=1, line_color="green")
fig.add_hline(y=level[4][1], line_width=1, line_color="red")
fig.add_hline(y=level[5][0], line_width=1, line_color="green")
fig.add_hline(y=level[5][1], line_width=1, line_color="red")



# Add a trace of the price for better clarity
fig.add_trace(go.Trace(
    x=btc.index,
    y=btc['Adj Close'],
    line_color="black",
    line_width=1
))

# Make it pretty
layout = go.Layout(
    plot_bgcolor='#efefef',
    showlegend=False,
    # Font Families
    font_family='Monospace',
    font_color='#000000',
    font_size=20,
    xaxis=dict(
        rangeslider=dict(
            visible=False
        ))
)
fig.update_layout(layout)
fig.show()

c:\Users\ASUS\.pyenv\pyenv-win\versions\3.10.0\lib\site-packages\plotly\graph_objs\_deprecations.py:687: DeprecationWarning:

plotly.graph_objs.Trace is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.




In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, timezone
import asyncio
import logging
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from collections import deque
import mplfinance as mpf
from matplotlib.widgets import Slider, CheckButtons
from concurrent.futures import ThreadPoolExecutor
import time
from sklearn.cluster import DBSCAN
from itertools import product
from joblib import Parallel, delayed
import pickle
import os
import sqlite3
from matplotlib.animation import FuncAnimation

# Configure logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
logging.getLogger('matplotlib').setLevel(logging.WARNING)
logging.getLogger('PIL').setLevel(logging.WARNING)

# Configuration
MT5_LOGIN = 240222940
MT5_PASSWORD = '55TTww$$'
MT5_SERVER = 'Exness-MT5Trial6'
SYMBOLS = ['EURUSD']
LIMIT = 2000

global DISPLAY_CANDLES
DISPLAY_CANDLES = 50

PLOT_TIMEFRAMES = ['H1']
AGGREGATE_TIMEFRAMES = ['M1', 'M5', 'M15']
SR_LEVELS_FILE = 'historical_sr_levels.pkl'
DB_FILE = 'ohlcv_cache.db'
DEFAULT_EPS = 0.0005
MIN_CANDLES = 14
UPDATE_INTERVAL = 1000  # ms
RETRY_DELAYS = [2, 4, 8]  # Seconds for exponential backoff

# Timeframe mapping
TIMEFRAME_MAP = {
    'M1': mt5.TIMEFRAME_M1, 'M5': mt5.TIMEFRAME_M5, 'M15': mt5.TIMEFRAME_M15,
    'M30': mt5.TIMEFRAME_M30, 'H1': mt5.TIMEFRAME_H1, 'H4': mt5.TIMEFRAME_H4, 'D1': mt5.TIMEFRAME_D1,
}
TIMEFRAME_MINUTES = {'M1': 1, 'M5': 5, 'M15': 15, 'M30': 30, 'H1': 60, 'H4': 240, 'D1': 1440}

# Global cache and connection state
ohlcv_cache = {}
grid_search_cache = {}
global_mt5_connected = False
historical_sr_levels = deque(maxlen=50)
fallback_data = {}  # Store last valid data per symbol/timeframe

# SQLite cache setup
def init_db():
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS ohlcv (
        symbol TEXT, timeframe TEXT, timestamp INTEGER, open REAL, high REAL, low REAL, close REAL, volume INTEGER,
        PRIMARY KEY (symbol, timeframe, timestamp)
    )''')
    conn.commit()
    conn.close()

# Class to store S/R levels
class SRLevel:
    def __init__(self, price, timestamp, is_support, touches=1, significance=1.0):
        self.price = price
        self.timestamp = timestamp
        self.is_support = is_support
        self.touches = touches
        self.significance = significance

    def update_touch(self):
        self.touches += 1
        self.significance += 0.2

# Initialize MetaTrader5
def initialize_mt5():
    global global_mt5_connected
    for attempt, delay in enumerate(RETRY_DELAYS, 1):
        try:
            if mt5.initialize(login=MT5_LOGIN, password=MT5_PASSWORD, server=MT5_SERVER, timeout=30000):
                global_mt5_connected = True
                for symbol in SYMBOLS:
                    if not mt5.symbol_select(symbol, True):
                        logger.error(f"Symbol {symbol} not available: {mt5.last_error()}")
                        mt5.shutdown()
                        global_mt5_connected = False
                        return False
                symbols = mt5.symbols_get()
                if not any(s.name == 'EURUSD' for s in symbols):
                    logger.error("EURUSD not found in market watch")
                    mt5.shutdown()
                    global_mt5_connected = False
                    return False
                logger.info("MT5 initialized successfully")
                return True
            else:
                logger.warning(f"MT5 initialization failed: {mt5.last_error()}")
        except Exception as e:
            logger.error(f"MT5 initialization exception (attempt {attempt}): {e}")
        time.sleep(delay)
    global_mt5_connected = False
    logger.error("Failed to initialize MT5 after retries")
    return False

def fetch_ohlcv_from_mt5(symbol, timeframe, limit, incremental=False):
    global global_mt5_connected
    try:
        start_time = time.time()
        if not global_mt5_connected or not mt5.terminal_info():
            logger.warning("MT5 connection lost, attempting to reconnect")
            if not initialize_mt5():
                return None
        timeframe_val = timeframe if isinstance(timeframe, int) else TIMEFRAME_MAP.get(timeframe, mt5.TIMEFRAME_M1)
        candles = 10 if incremental else limit
        now = datetime.now(timezone.utc)
        minutes_per_candle = TIMEFRAME_MINUTES.get(timeframe if isinstance(timeframe, str) else 'M1', 1)
        from_date = now - timedelta(minutes=candles * minutes_per_candle)
        rates = mt5.copy_rates_range(symbol, timeframe_val, from_date, now)
        if rates is None or len(rates) == 0:
            logger.warning(f"copy_rates_range failed for {symbol} on {timeframe}, trying copy_rates_from_pos")
            rates = mt5.copy_rates_from_pos(symbol, timeframe_val, 0, candles)
        if rates is None or len(rates) == 0:
            logger.error(f"No data fetched for {symbol} on {timeframe}: {mt5.last_error()}")
            return None
        df = pd.DataFrame(rates)
        df['time'] = pd.to_datetime(df['time'], unit='s', utc=True)
        df = df.rename(columns={
            'time': 'timestamp', 'open': 'open', 'high': 'high',
            'low': 'low', 'close': 'close', 'tick_volume': 'volume'
        })
        df = df[['timestamp', 'open', 'high', 'low', 'close', 'volume']]
        if df.empty or len(df) < MIN_CANDLES:
            logger.error(f"Empty or insufficient DataFrame ({len(df)} candles) for {symbol} on {timeframe}")
            return None
        df.set_index('timestamp', inplace=True)
        df = df.sort_index()
        df = df.astype({'open': 'float32', 'high': 'float32', 'low': 'float32', 'close': 'float32', 'volume': 'int32'})
        df = df.ffill().bfill().dropna()
        if df[['open', 'high', 'low', 'close']].le(0).any().any():
            logger.error(f"Invalid price data (zero or negative) for {symbol} on {timeframe}")
            return None
        if df['volume'].le(0).any():
            logger.warning(f"Zero volume detected for {symbol} on {timeframe}, setting to 1")
            df['volume'] = df['volume'].clip(lower=1)
        logger.debug(f"Fetched {len(df)} candles for {symbol} on {timeframe}, took {time.time() - start_time:.3f}s")
        fallback_data[(symbol, timeframe if isinstance(timeframe, str) else 'M1')] = df
        return df
    except Exception as e:
        logger.error(f"Error fetching OHLCV for {symbol} on {timeframe}: {e}, MT5 error: {mt5.last_error()}")
        global_mt5_connected = False
        return None

def cache_to_db(symbol, timeframe, df):
    conn = sqlite3.connect(DB_FILE)
    df.reset_index().to_sql('ohlcv', conn, if_exists='append', index=False, method='multi')
    conn.commit()
    conn.close()

def fetch_from_db(symbol, timeframe, limit):
    conn = sqlite3.connect(DB_FILE)
    query = f"""
        SELECT * FROM ohlcv
        WHERE symbol = ? AND timeframe = ?
        ORDER BY timestamp DESC
        LIMIT ?
    """
    df = pd.read_sql_query(query, conn, params=(symbol, timeframe, limit))
    conn.close()
    if df.empty:
        return None
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.set_index('timestamp', inplace=True)
    df = df[['open', 'high', 'low', 'close', 'volume']].astype({
        'open': 'float32', 'high': 'float32', 'low': 'float32', 'close': 'float32', 'volume': 'int32'
    })
    return df.sort_index()

def fetch_ohlcv(symbol, timeframe, limit, retries=3, incremental=False):
    cache_key = f"{symbol}_{timeframe}_{limit}"
    cache_expiry = {'M1': 60, 'M5': 300, 'M15': 900, 'M30': 1800, 'H1': 3600, 'H4': 14400, 'D1': 86400}
    if cache_key in ohlcv_cache:
        cached_df, cache_time = ohlcv_cache[cache_key]
        if (datetime.now(timezone.utc) - cache_time).total_seconds() < cache_expiry.get(timeframe, 60):
            return cached_df
    df_db = fetch_from_db(symbol, timeframe, limit)
    if df_db is not None and len(df_db) >= MIN_CANDLES:
        ohlcv_cache[cache_key] = (df_db, datetime.now(timezone.utc))
        return df_db
    for attempt in range(retries):
        df = fetch_ohlcv_from_mt5(symbol, timeframe, limit, incremental)
        if df is not None:
            cache_to_db(symbol, timeframe, df)
            ohlcv_cache[cache_key] = (df, datetime.now(timezone.utc))
            return df
        logger.warning(f"Attempt {attempt + 1} failed for {symbol}")
        time.sleep(RETRY_DELAYS[attempt])
    logger.error(f"Failed to fetch data for {symbol} after {retries} attempts")
    if (symbol, timeframe) in fallback_data:
        logger.warning(f"Using fallback data for {symbol} on {timeframe}")
        return fallback_data[(symbol, timeframe)]
    df_db = fetch_from_db(symbol, timeframe, limit)
    if df_db is not None:
        logger.warning(f"Using SQLite cached data for {symbol} on {timeframe}")
        return df_db
    return None

def calculate_fibonacci_levels(df, initial_candles):
    data = df[-initial_candles:]
    if len(data) < 2:
        logger.warning("Insufficient data for Fibonacci levels")
        return []
    swing_high = data['high'].max()
    swing_low = data['low'].min()
    diff = swing_high - swing_low
    if diff <= 0:
        logger.warning("Invalid swing range for Fibonacci levels")
        return []
    return [swing_low + diff * level for level in [0.0, 0.236, 0.382, 0.5, 0.618, 0.786, 1.0]]

def calculate_macd(df, fast=12, slow=26, signal=9):
    data = df.copy()
    if len(data) < max(fast, slow, signal):
        logger.warning("Insufficient data for MACD")
        return False
    data['ema_fast'] = data['close'].ewm(span=fast, adjust=False).mean()
    data['ema_slow'] = data['close'].ewm(span=slow, adjust=False).mean()
    data['macd'] = data['ema_fast'] - data['ema_slow']
    data['signal'] = data['macd'].ewm(span=signal, adjust=False).mean()
    return data['macd'].iloc[-1] > data['signal'].iloc[-1]

def calculate_rsi(df, period=14):
    data = df.copy()
    if len(data) < period:
        logger.warning("Insufficient data for RSI")
        return 50.0
    delta = data['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rs = rs.replace([np.inf, -np.inf], np.nan).fillna(1.0)
    rsi = 100 - (100 / (1 + rs))
    return rsi.iloc[-1]

def calculate_bollinger_bands(df, period=20, std_dev=2):
    data = df.copy()
    if len(data) < period:
        logger.warning("Insufficient data for Bollinger Bands")
        return None, None, None
    data['sma'] = data['close'].rolling(window=period).mean()
    data['std'] = data['close'].rolling(window=period).std()
    data['upper'] = data['sma'] + (data['std'] * std_dev)
    data['lower'] = data['sma'] - (data['std'] * std_dev)
    return data['sma'].iloc[-1], data['upper'].iloc[-1], data['lower'].iloc[-1]

def calculate_atr(df, period=14, timeframe='M1'):
    data = df.copy()
    if len(data) < period or data.empty:
        logger.warning(f"Insufficient data ({len(data)} candles) for ATR, returning default: {DEFAULT_EPS}")
        return DEFAULT_EPS
    data['tr'] = np.maximum(
        data['high'] - data['low'],
        np.maximum(
            abs(data['high'] - data['close'].shift()),
            abs(data['low'] - data['close'].shift())
        )
    )
    data['tr'] = data['tr'].clip(lower=DEFAULT_EPS / 10)
    data['tr'] = data['tr'].fillna(DEFAULT_EPS / 10)
    atr = data['tr'].rolling(window=period).mean().iloc[-1]
    if np.isnan(atr) or atr <= 0:
        logger.warning(f"Invalid ATR ({atr}), returning default: {DEFAULT_EPS}")
        return DEFAULT_EPS
    timeframe_multipliers = {'M1': 0.5, 'M5': 0.75, 'M15': 1.0}
    atr *= timeframe_multipliers.get(timeframe, 1.0)
    logger.debug(f"Calculated ATR: {atr} (timeframe: {timeframe})")
    return atr

def calculate_volume_profile(df, bins=50):
    data = df.copy()
    if len(data) < MIN_CANDLES:
        logger.warning("Insufficient data for volume profile")
        return [], []
    price_range = data['high'].max() - data['low'].min()
    if price_range <= 0:
        logger.warning("Invalid price range for volume profile")
        return [], []
    bin_edges = np.linspace(data['low'].min(), data['high'].max(), bins + 1)
    volumes = np.zeros(bins)
    for i in range(len(data)):
        price = data['close'].iloc[i]
        volume = data['volume'].iloc[i]
        bin_idx = np.digitize(price, bin_edges) - 1
        if 0 <= bin_idx < bins:
            volumes[bin_idx] += volume
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    threshold = np.percentile(volumes, 80)
    support_levels = bin_centers[volumes > threshold]
    resistance_levels = support_levels
    return support_levels.tolist(), resistance_levels.tolist()

def calculate_pivot_points(df, initial_candles):
    data = df[-initial_candles:].copy()
    if len(data) < 2:
        logger.warning("Insufficient data for pivot points")
        return [], []
    pivot = (data['high'].max() + data['low'].min() + data['close'].iloc[-1]) / 3
    support = 2 * pivot - data['high'].max()
    resistance = 2 * pivot - data['low'].min()
    return [support], [resistance]

def optimize_snr_parameters(df, cache_key, use_grid_search=True, timeframe='M1'):
    if not use_grid_search:
        return DEFAULT_EPS, 2
    cache_file = f"{cache_key}.pkl"
    if os.path.exists(cache_file):
        with open(cache_file, 'rb') as f:
            params, cache_time = pickle.load(f)
        if (datetime.now(timezone.utc) - cache_time).total_seconds() < 3600:
            return params
    atr = calculate_atr(df, timeframe=timeframe)
    eps_range = [atr * x for x in [0.5, 1.0]] if not np.isnan(atr) else [DEFAULT_EPS]
    min_samples_range = [2, 3]
    def evaluate_params(eps, min_samples):
        temp_levels = deque(maxlen=50)
        support_snr_line, resistance_snr_line = calculate_snr_line(
            df, initial_candles=50, eps=eps, min_samples=min_samples
        )
        update_historical_sr_levels(df, support_snr_line, resistance_snr_line, temp_levels)
        return backtest_sr_levels(df, temp_levels), (eps, min_samples)
    results = Parallel(n_jobs=-1)(
        delayed(evaluate_params)(eps, min_samples)
        for eps, min_samples in product(eps_range, min_samples_range)
    )
    best_accuracy = 0
    best_params = (DEFAULT_EPS, 2)
    for accuracy, (eps, min_samples) in results:
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = (eps, min_samples)
        logger.debug(f"Tested eps={eps:.6f}, min_samples={min_samples}, accuracy={accuracy:.2%}")
        if accuracy > 0.9:
            best_params = (eps, min_samples)
            break
    logger.info(f"Best parameters: eps={best_params[0]:.6f}, min_samples={best_params[1]}, accuracy={best_accuracy:.2%}")
    with open(cache_file, 'wb') as f:
        pickle.dump((best_params, datetime.now(timezone.utc)), f)
    return best_params

def calculate_snr_line(df, initial_candles, eps=DEFAULT_EPS, min_samples=2, use_volume_profile=False):
    data = df[-initial_candles:].copy()
    if len(data) < MIN_CANDLES or data.empty:
        logger.warning(f"Insufficient data ({len(data)} candles) for SNR, using pivot points")
        support_levels, resistance_levels = calculate_pivot_points(df, initial_candles)
        if not support_levels or not resistance_levels:
            current_price = data['close'].iloc[-1] if not data.empty else 1.0
            x = np.arange(max(len(data), 1))
            return np.full(len(x), current_price - DEFAULT_EPS), np.full(len(x), current_price + DEFAULT_EPS)
        return np.full(len(data), support_levels[0]), np.full(len(data), resistance_levels[0])
    
    if use_volume_profile:
        support_levels, resistance_levels = calculate_volume_profile(data)
        if not support_levels or not resistance_levels:
            logger.warning("Volume profile failed, using pivot points")
            support_levels, resistance_levels = calculate_pivot_points(data, initial_candles)
            if not support_levels or not resistance_levels:
                current_price = data['close'].iloc[-1]
                x = np.arange(len(data))
                return np.full(len(data), current_price - DEFAULT_EPS), np.full(len(data), current_price + DEFAULT_EPS)
            return np.full(len(data), support_levels[0]), np.full(len(data), resistance_levels[0])
        current_price = data['close'].iloc[-1]
        support_price = min(support_levels, key=lambda x: abs(x - current_price))
        resistance_price = min(resistance_levels, key=lambda x: abs(x - current_price))
        return np.full(len(data), support_price), np.full(len(data), resistance_price)
    
    data['high_ema'] = data['high'].ewm(span=15, adjust=False).mean()
    data['low_ema'] = data['low'].ewm(span=15, adjust=False).mean()
    
    is_bullish = calculate_macd(data)
    rsi = calculate_rsi(data)
    quantile = 0.75 if rsi > 50 else 0.70
    high_quantile = quantile if is_bullish else quantile + 0.05
    low_quantile = 1 - quantile if is_bullish else 1 - (quantile + 0.05)
    
    atr = calculate_atr(data)
    logger.debug(f"ATR: {atr}, Data length: {len(data)}, High EMA: {data['high_ema'].iloc[-1]}, Low EMA: {data['low_ema'].iloc[-1]}")
    high_threshold = data['high_ema'].quantile(high_quantile, interpolation='midpoint') + atr * 0.2
    low_threshold = data['low_ema'].quantile(low_quantile, interpolation='midpoint') - atr * 0.2
    
    volume_weights = data['volume'] / data['volume'].max()
    highs = data['high_ema'][data['high_ema'] >= high_threshold].values
    lows = data['low_ema'][data['low_ema'] <= low_threshold].values
    high_volumes = volume_weights[data['high_ema'] >= high_threshold].values
    low_volumes = volume_weights[data['low_ema'] <= low_threshold].values
    
    if len(highs) == 0 or len(lows) == 0:
        logger.warning(f"No valid highs ({len(highs)}) or lows ({len(lows)}) for clustering, using pivot points")
        support_levels, resistance_levels = calculate_pivot_points(data, initial_candles)
        if not support_levels or not resistance_levels:
            current_price = data['close'].iloc[-1]
            x = np.arange(len(data))
            return np.full(len(data), current_price - atr * 0.5), np.full(len(data), current_price + atr * 0.5)
        return np.full(len(data), support_levels[0]), np.full(len(data), resistance_levels[0])
    
    highs = highs.reshape(-1, 1)
    lows = lows.reshape(-1, 1)
    db_highs = DBSCAN(eps=eps, min_samples=min_samples).fit(highs, sample_weight=high_volumes)
    db_lows = DBSCAN(eps=eps, min_samples=min_samples).fit(lows, sample_weight=low_volumes)
    labels_highs = db_highs.labels_
    labels_lows = db_lows.labels_
    
    resistance_levels = [highs[labels_highs == label].mean() for label in set(labels_highs) if label != -1]
    support_levels = [lows[labels_lows == label].mean() for label in set(labels_lows) if label != -1]
    
    current_price = data['close'].iloc[-1]
    resistance_price = min(resistance_levels, key=lambda x: abs(x - current_price)) if resistance_levels else high_threshold
    support_price = min(support_levels, key=lambda x: abs(x - current_price)) if support_levels else low_threshold
    
    x = np.arange(len(data))
    support_snr_line = np.full(len(data), support_price)
    resistance_snr_line = np.full(len(data), resistance_price)
    
    return support_snr_line, resistance_snr_line

def find_pivots(data, price_col, is_high=True):
    prices = data[price_col].values
    if len(prices) < 3:
        logger.warning("Insufficient data for pivot detection")
        return 0, 1
    x = np.arange(len(prices))
    coeffs = np.polyfit(x, prices, 2)
    y_pred = np.polyval(coeffs, x)
    deviations = prices - y_pred
    
    if is_high:
        pivot_idx = deviations.argmax()
        deviations[pivot_idx] = -np.inf
        second_pivot_idx = deviations.argmax()
    else:
        pivot_idx = deviations.argmin()
        deviations[pivot_idx] = np.inf
        second_pivot_idx = deviations.argmin()
    
    return pivot_idx, second_pivot_idx

def validate_trend_line(df, trend_line, is_support=True):
    data = df.copy()
    prices = data['low' if is_support else 'high']
    touches = np.sum(np.abs(prices - trend_line) < 0.0005)
    return touches >= 3

def calculate_trend_line(df, initial_candles):
    from sklearn.linear_model import RANSACRegressor
    data = df[-initial_candles:].copy()
    if len(data) < 3:
        logger.warning("Insufficient data for trend line, using flat lines")
        x = np.arange(max(len(data), 1))
        low = data['low'].min() if not data.empty else 1.0
        high = data['high'].max() if not data.empty else 1.0
        return np.full(len(x), low), np.full(len(x), high)
    
    x = np.arange(len(data)).reshape(-1, 1)
    atr = calculate_atr(data)
    
    high_pivot, high_second_pivot = find_pivots(data, 'high', is_high=True)
    low_pivot, low_second_pivot = find_pivots(data, 'low', is_high=False)
    
    high_prices = data['high'].values + atr * 0.2
    low_prices = data['low'].values - atr * 0.2
    
    ransac_support = RANSACRegressor().fit(x[[low_pivot, low_second_pivot]], low_prices[[low_pivot, low_second_pivot]])
    ransac_resistance = RANSACRegressor().fit(x[[high_pivot, high_second_pivot]], high_prices[[high_pivot, high_second_pivot]])
    
    support_trend_line = ransac_support.predict(x)
    resistance_trend_line = ransac_resistance.predict(x)
    
    if not validate_trend_line(data, support_trend_line, is_support=True):
        logger.warning("Support trend line not validated, using polynomial fit")
        support_coeffs = np.polyfit(np.arange(len(data)), data['low'].values, 2)
        support_trend_line = np.polyval(support_coeffs, np.arange(len(data)))
    if not validate_trend_line(data, resistance_trend_line, is_support=False):
        logger.warning("Resistance trend line not validated, using polynomial fit")
        resistance_coeffs = np.polyfit(np.arange(len(data)), data['high'].values, 2)
        resistance_trend_line = np.polyval(resistance_coeffs, np.arange(len(data)))
    
    return support_trend_line, resistance_trend_line

def backtest_sr_levels(df, historical_sr_levels):
    accuracy = 0
    total_tests = 0
    prices = df['close'].values
    for i in range(1, len(df)):
        price = prices[i]
        for level in historical_sr_levels:
            if abs(price - level.price) < 0.0005:
                prev_price = prices[i-1]
                next_price = prices[i+1] if i+1 < len(df) else price
                if (prev_price > level.price and next_price < level.price) or \
                   (prev_price < level.price and next_price > level.price):
                    accuracy += 1
                total_tests += 1
    return accuracy / total_tests if total_tests > 0 else 0

def generate_trading_signal(df, support_snr_line, resistance_snr_line, support_trend_line, resistance_trend_line):
    if len(df) < 2:
        return "Hold"
    current_price = df['close'].iloc[-1]
    atr = calculate_atr(df)
    sma, upper_bb, lower_bb = calculate_bollinger_bands(df)
    if sma is None:
        return "Hold"
    if (abs(current_price - support_snr_line[-1]) < atr * 0.1 and
        current_price > support_trend_line[-1] and
        current_price < sma and current_price > lower_bb):
        return "Buy"
    elif (abs(current_price - resistance_snr_line[-1]) < atr * 0.1 and
          current_price < resistance_trend_line[-1] and
          current_price > sma and current_price < upper_bb):
        return "Sell"
    return "Hold"

def update_historical_sr_levels(df, support_snr_line, resistance_snr_line, levels=None, show_fib=True):
    if levels is None:
        levels = historical_sr_levels
    latest_time = df.index[-1] if not df.empty else datetime.now(timezone.utc)
    support_price = support_snr_line[-1]
    resistance_price = resistance_snr_line[-1]
    
    price_diff = 0.0005
    if show_fib:
        fib_levels = calculate_fibonacci_levels(df, initial_candles=50)
        for level in fib_levels:
            if not any(abs(level - sr.price) < price_diff for sr in levels):
                levels.append(SRLevel(level, latest_time, True, significance=0.5))
    
    for level in levels:
        age_hours = (latest_time - level.timestamp).total_seconds() / 3600
        level.significance *= np.exp(-age_hours / 24)
        if level.is_support and abs(level.price - support_price) < price_diff:
            level.update_touch()
            return
        if not level.is_support and abs(level.price - resistance_price) < price_diff:
            level.update_touch()
            return
    
    levels.append(SRLevel(support_price, latest_time, True))
    levels.append(SRLevel(resistance_price, latest_time, False))

def save_sr_levels(levels, filename=SR_LEVELS_FILE):
    with open(filename, 'wb') as f:
        pickle.dump(list(levels), f)
    logger.info(f"Saved S/R levels to {filename}")

def load_sr_levels(filename=SR_LEVELS_FILE):
    if os.path.exists(filename):
        with open(filename, 'rb') as f:
            levels = pickle.load(f)
        logger.info(f"Loaded S/R levels from {filename}")
        return deque(levels, maxlen=50)
    return deque(maxlen=50)

def aggregate_sr_levels(symbol, timeframes, limit):
    all_levels = []
    timeframe_weights = {'M1': 0.2, 'M5': 0.3, 'M15': 0.5}
    for timeframe in timeframes:
        df = fetch_ohlcv(symbol, timeframe, limit)
        if df is None:
            logger.warning(f"Skipping {timeframe} for {symbol} due to fetch failure")
            continue
        support_snr_line, resistance_snr_line = calculate_snr_line(df, initial_candles=50)
        weight = timeframe_weights.get(timeframe, 1.0 / len(timeframes))
        all_levels.append(SRLevel(support_snr_line[-1], df.index[-1], True, significance=weight))
        all_levels.append(SRLevel(resistance_snr_line[-1], df.index[-1], False, significance=weight))
    if not all_levels:
        logger.warning(f"No valid data for {symbol}, using M1 pivot points")
        df_m1 = fetch_ohlcv(symbol, 'M1', limit)
        if df_m1 is not None:
            support_snr_line, resistance_snr_line = calculate_snr_line(df_m1, initial_candles=50)
            all_levels.append(SRLevel(support_snr_line[-1], df_m1.index[-1], True, significance=0.2))
            all_levels.append(SRLevel(resistance_snr_line[-1], df_m1.index[-1], False, significance=0.2))
    merged_levels = []
    price_diff = 0.0005
    for level in all_levels:
        matched = False
        for merged in merged_levels:
            if abs(level.price - merged.price) < price_diff and level.is_support == merged.is_support:
                merged.significance += level.significance
                merged.touches += 1
                matched = True
                break
        if not matched:
            merged_levels.append(level)
    return merged_levels

async def plot(plot_df, symbol, timeframe, snr_candles=DISPLAY_CANDLES, trend_candles=DISPLAY_CANDLES, backtest_index=0):
    if plot_df is None or len(plot_df) < DISPLAY_CANDLES:
        logger.error(f"Insufficient data ({len(plot_df) if plot_df is not None else 0} candles) for plotting")
        return
    
    cache_key = f"{symbol}_{timeframe}"
    fig = plt.figure(figsize=(12, 8))
    gs = fig.add_gridspec(7, 1, height_ratios=[4, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2], hspace=0.5)
    ax_candles = fig.add_subplot(gs[0])
    
    ax_snr = fig.add_axes([0.15, 0.35, 0.65, 0.04])
    ax_candles_s = fig.add_axes([0.15, 0.30, 0.65, 0.04])
    ax_trend = fig.add_axes([0.15, 0.25, 0.65, 0.04])
    ax_backtest = fig.add_axes([0.15, 0.20, 0.65, 0.04])
    ax_grid = fig.add_axes([0.15, 0.15, 0.65, 0.04])
    ax_fib = fig.add_axes([0.15, 0.10, 0.65, 0.04])
    
    snr_slider = Slider(ax_snr, 'SNR Candles', 10, 200, valinit=snr_candles, valstep=1)
    candles_slider = Slider(ax_candles_s, 'Candles', 10, 200, valinit=DISPLAY_CANDLES, valstep=1)
    trend_slider = Slider(ax_trend, 'Trend Candles', 10, 200, valinit=trend_candles, valstep=1)
    backtest_slider = Slider(ax_backtest, 'Backtest', 0, len(plot_df) - DISPLAY_CANDLES, valinit=backtest_index, valstep=1)
    grid_check = CheckButtons(ax_grid, ['Grid Search', 'Volume Profile'], [True, False])
    fib_check = CheckButtons(ax_fib, ['Fibonacci'], [True])
    
    plot_cache = {'support_snr': None, 'resistance_snr': None, 'support_trend': None, 'resistance_trend': None}
    
    def update(frame=None):
        global DISPLAY_CANDLES
        nonlocal plot_df
        try:
            ax_candles.clear()
            
            new_df = fetch_ohlcv(symbol, timeframe, LIMIT, incremental=True)
            if new_df is not None:
                plot_df = pd.concat([plot_df, new_df]).drop_duplicates().sort_index()[-LIMIT:]
            
            if plot_df is None or len(plot_df) < DISPLAY_CANDLES:
                logger.error(f"Plot data invalid ({len(plot_df) if plot_df is not None else 0} candles)")
                return
            
            snr_candles = int(snr_slider.val)
            candles = int(candles_slider.val)
            trend_candles = int(trend_slider.val)
            backtest_index = int(backtest_slider.val)
            use_grid_search, use_volume_profile = grid_check.get_status()
            show_fib = fib_check.get_status()[0]
            
            DISPLAY_CANDLES = candles
            
            start_idx = backtest_index
            end_idx = start_idx + DISPLAY_CANDLES
            if end_idx > len(plot_df):
                end_idx = len(plot_df)
                start_idx = max(0, end_idx - DISPLAY_CANDLES)
            display_df = plot_df.iloc[start_idx:end_idx]
            
            snr_data = plot_df.iloc[max(0, backtest_index - snr_candles):end_idx]
            trend_data = plot_df.iloc[max(0, backtest_index - trend_candles):end_idx]
            
            best_eps, best_min_samples = optimize_snr_parameters(snr_data, cache_key, use_grid_search, timeframe)
            support_snr_line, resistance_snr_line = calculate_snr_line(
                snr_data, min(snr_candles, len(snr_data)), eps=best_eps, min_samples=best_min_samples,
                use_volume_profile=use_volume_profile
            )
            support_trend_line, resistance_trend_line = calculate_trend_line(trend_data, min(trend_candles, len(trend_data)))
            
            update_historical_sr_levels(plot_df, support_snr_line, resistance_snr_line, show_fib=show_fib)
            sr_accuracy = backtest_sr_levels(plot_df, historical_sr_levels)
            signal = generate_trading_signal(plot_df, support_snr_line, resistance_snr_line, support_trend_line, resistance_trend_line)
            
            plot_cache['support_snr'] = support_snr_line[-len(display_df):]
            plot_cache['resistance_snr'] = resistance_snr_line[-len(display_df):]
            plot_cache['support_trend'] = support_trend_line[-len(display_df):]
            plot_cache['resistance_trend'] = resistance_trend_line[-len(display_df):]
            
            addplots = [
                mpf.make_addplot(plot_cache['support_snr'], color='green', width=1.5, ax=ax_candles),
                mpf.make_addplot(plot_cache['resistance_snr'], color='red', width=1.5, ax=ax_candles),
                mpf.make_addplot(plot_cache['support_trend'], color='blue', width=1.5, ax=ax_candles),
                mpf.make_addplot(plot_cache['resistance_trend'], color='orange', width=1.5, ax=ax_candles)
            ]
            
            sma, upper_bb, lower_bb = calculate_bollinger_bands(display_df)
            if sma is not None:
                sma_line = np.full(len(display_df), sma)
                upper_bb_line = np.full(len(display_df), upper_bb)
                lower_bb_line = np.full(len(display_df), lower_bb)
                addplots.extend([
                    mpf.make_addplot(sma_line, color='purple', width=1, ax=ax_candles),
                    mpf.make_addplot(upper_bb_line, color='purple', width=1, linestyle='--', ax=ax_candles),
                    mpf.make_addplot(lower_bb_line, color='purple', width=1, linestyle='--', ax=ax_candles)
                ])
            
            for level in historical_sr_levels:
                if level.timestamp >= display_df.index[0] and level.timestamp <= display_df.index[-1]:
                    level_line = np.full(len(display_df), level.price)
                    color = 'darkgreen' if level.is_support else 'darkred'
                    addplots.append(mpf.make_addplot(level_line, color=color, width=0.8, linestyle='--', ax=ax_candles))
                    ax_candles.annotate(
                        f"{'S' if level.is_support else 'R'}:{level.price:.5f} ({level.touches})",
                        xy=(display_df.index[-1], level.price),
                        xytext=(5, 0),
                        textcoords="offset points",
                        color=color
                    )
            
            atr = calculate_atr(plot_df, timeframe=timeframe)
            price_range = display_df['high'].max() - display_df['low'].min()
            ax_candles.set_ylim(
                display_df['low'].min() - max(price_range * 0.15, atr * 0.5),
                display_df['high'].max() + max(price_range * 0.15, atr * 0.5)
            )
            
            param_text = f"eps={best_eps:.6f}, min_samples={best_min_samples}, Signal={signal}"
            mpf.plot(
                display_df,
                type='candle',
                title=f"{symbol} - {timeframe} (S/R Accuracy: {sr_accuracy:.2%}, {param_text})",
                ylabel='Price',
                addplot=addplots,
                style='classic',
                show_nontrading=False,
                ax=ax_candles,
            )
            
            save_sr_levels(historical_sr_levels)
            fig.canvas.draw_idle()
            fig.canvas.flush_events()
        except Exception as e:
            logger.error(f"Error in plot update: {e}")
    
    snr_slider.on_changed(update)
    candles_slider.on_changed(update)
    trend_slider.on_changed(update)
    backtest_slider.on_changed(update)
    grid_check.on_clicked(update)
    fib_check.on_clicked(update)
    
    ani = FuncAnimation(fig, update, interval=UPDATE_INTERVAL, cache_frame_data=False)
    
    try:
        update(None)
        plt.show()
    except Exception as e:
        logger.error(f"Initial plot failed: {e}")
        raise

def fetch_all_data(symbols, timeframes, limit):
    with ThreadPoolExecutor() as executor:
        futures = [
            executor.submit(fetch_ohlcv, symbol, timeframe, limit)
            for symbol in symbols for timeframe in timeframes
        ]
        results = [f.result() for f in futures]
    return results, [(symbol, timeframe) for symbol in symbols for timeframe in timeframes]

async def main():
    init_db()
    if not initialize_mt5():
        logger.error("Failed to initialize MT5, exiting")
        return
    
    global historical_sr_levels
    historical_sr_levels = load_sr_levels()
    
    for symbol in SYMBOLS:
        historical_sr_levels.extend(aggregate_sr_levels(symbol, AGGREGATE_TIMEFRAMES, LIMIT))
        data, metadata = fetch_all_data([symbol], PLOT_TIMEFRAMES, LIMIT)
        for df, (_, timeframe) in zip(data, metadata):
            if df is not None:
                try:
                    await plot(df, symbol, timeframe)
                except Exception as e:
                    logger.error(f"Plotting failed for {symbol} on {timeframe}: {e}")
    
    mt5.shutdown()

if __name__ == '__main__':
    asyncio.run(main())